### Crawl Vietnam Stock Data

This project is to crawl Vietnam stock data (around 1600 companies) everyday for further analyst and investment

In this project, I will use some libraries: Selenium (interact with website to download data), Pandas,Openpyxl (clean & consolidate data) & Pyodbc (update data to SQL database for further analyst)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import os
import pandas as pd
import openpyxl
import pyodbc
from time import sleep
from datetime import datetime
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [6]:
#open website
driver = webdriver.Chrome(executable_path=r"F:\Data\Funix\Chromedriver.exe")
url = 'https://finance.vietstock.vn/ket-qua-giao-dich?tab=thong-ke-gia&exchange=3'
driver.get(url)

In [7]:
#login account
with open('F:/Data/Funix/Passcode.txt') as f:
    lines = []
    for line in f:
        lines.append(line.rstrip())
username = lines[0]
password = lines[1]

Dang_nhap = driver.find_element(by='xpath', value='/html/body/div[2]/div[6]/div/div[2]/div[2]/a[3]').click()
Ten_email = driver.find_element(by='id', value='txtEmailLogin')
Ten_email.clear()
Ten_email.send_keys(username)
Ten_matkhau = driver.find_element(by='id', value='txtPassword')
Ten_matkhau.clear()
Ten_matkhau.send_keys(password)
driver.find_element(by='id', value='btnLoginAccount').submit()


In [8]:
#Download file
From_date = datetime.today().strftime('%d/%m/%Y')
#From_date = '22/04/2022'
san_list = ['HOSE','HNX','UPCoM']


for san in san_list:
    san_field = driver.find_element(by='xpath', value='//*[@id="trading-result"]/div/div[1]/div[1]/div/div[1]/div/div[1]/div/select')
    san_field.send_keys(san)
    Fromdate_field = driver.find_element(by='xpath', value='//*[@id="txtFromDate"]/input')
    Fromdate_field.clear()
    Fromdate_field.send_keys(From_date)
    Todate_field = driver.find_element(by='xpath', value='//*[@id="txtToDate"]/input')
    Todate_field.clear()
    Todate_field.send_keys(From_date)
    driver.find_element(by='xpath', value='//*[@id="trading-result"]/div/div[1]/div[1]/div/div[2]/button').click()
    driver.find_element(by='xpath', value='//*[@id="trading-result"]/div/div[1]/div[1]/div/div[2]/a').click()




In [9]:
#Consolidate file
path = 'C:/Users/vinhbui/Downloads/'
list_file = []


for file in os.listdir(path):
    if file.startswith("KQGD"):
        list_file.append(file)

dict = {2: 'Ngay',3: 'Ma',4: 'ThamChieu',5: 'MoCua',6: 'DongCua',7: 'CaoNhat',8: 'ThapNhat',9: 'TrungBinh',10: 'ThayDoiGiaTien',11: 'ThayDoiGiaPhanTram',12: 'GdKhopLenh',13: 'GtKhopLenh',14: 'GdThoaThuan',15: 'GtThoaThuan',16: 'TongKlGiaoDich',17: 'TongGtGiaoDich',18: 'VonHoaThiTruong'}
ds = list(range(2,19))
frames = []

for file in list_file:
    filepath = path + file
    df1 = pd.read_excel(filepath, skiprows = 16, header = None, usecols = ds)
    frames.append(df1)
    result = pd.concat(frames)
 

path2 = 'C:/Users/vinhbui/Downloads/'
df = result
df.rename(columns=dict, inplace=True)
df.to_excel(path2 + 'DataStockConsolidate.xlsx', index=False)

In [10]:
#Import consolidated file to SQL

df1 = pd.read_excel('C:/Users/vinhbui/Downloads/DataStockConsolidate.xlsx')
server = 'PC-VINHBUI' 
db1 = 'StockProject' 
tcon = 'yes'
cnxn = pyodbc.connect(driver='{SQL Server Native Client 11.0}', host=server, database=db1,trusted_connection=tcon)
cursor = cnxn.cursor()

# Insert Dataframe into SQL Server:
for index, row in df1.iterrows():
     cursor.execute("INSERT INTO Stockdata values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",  row.Ngay, row.Ma, row.ThamChieu, row.MoCua, row.DongCua, row.CaoNhat, row.ThapNhat, row.TrungBinh, row.ThayDoiGiaTien, row.ThayDoiGiaPhanTram, row.GdKhopLenh, row.GtKhopLenh, row.GdThoaThuan, row.GtThoaThuan, row.TongKlGiaoDich, row.TongGtGiaoDich, row.VonHoaThiTruong,)
cnxn.commit()
cursor.close()

In [11]:
#Delete file after import
path = 'C:/Users/vinhbui/Downloads/'
os.listdir(path)

list_file_delete = []

for file in os.listdir(path):
    if file.startswith("KQGD") or file.startswith("DataStock"):
        os.remove(path + file)